In [ ]:
from mlflow import MlflowClient
from mlflow.entities import ViewType

from bugbug.trackers.mlflow_config import tracking_uri

run = MlflowClient(registry_uri="gs://oneclick-mlflow-store-f8bf20e5c6", tracking_uri=tracking_uri).search_runs(
    experiment_ids="2",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=1,
    order_by=["metrics.test_accuracy_mean DESC"],
)
print(run[0].data.metrics["test_accuracy_mean"])

**Run inference on model from Registry**

In [3]:
import mlflow
import numpy as np
import random

model_name = "CLF Model"
model_version = 1


# model can be loaded either way
model = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version}")

data = [random.uniform(0, 1) for a in range(26467)]

print(model.predict(np.array([data])))

[1]


In [19]:
model_name = "SpambugFull"
model_version = 1

In [20]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

2023/08/03 15:37:50 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.2.2, required: mlflow==2.5.0)
 - numpy (current: 1.22.4, required: numpy==1.23.2)
 - pyopenssl (current: 22.0.0, required: pyopenssl==23.2.0)
 - pytz (current: 2022.1, required: pytz==2023.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [21]:
print(model)

mlflow.pyfunc.loaded_model:
  artifact_path: spambug_full_model
  flavor: mlflow.pyfunc.model
  run_id: 99e12088cfe84d48961a9b597e1070f7



In [43]:
import json
with open("./tests/fixtures/bugs.json") as f:
    example_bugs = [json.loads(line) for line in f]
for bug in example_bugs:
    bug["filed_via"] = "bugzilla"
prediction_input = [json.dumps(e) for e in example_bugs[:3]]


In [46]:
## Not working
model.predict(prediction_input)

TypeError: the JSON object must be str, bytes or bytearray, not int

In [17]:
mlflow.pyfunc.get_model_dependencies(model_uri=f"models:/{model_name}/{model_version}")

2023/08/02 16:24:25 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /var/folders/jq/lt0dyzf14y93d8k18k68_4fm0000gn/T/tmp2r21_wzn/requirements.txt'.


'/var/folders/jq/lt0dyzf14y93d8k18k68_4fm0000gn/T/tmp2r21_wzn/requirements.txt'

In [18]:
!pip install -r /var/folders/jq/lt0dyzf14y93d8k18k68_4fm0000gn/T/tmp2r21_wzn/requirements.txt

You should consider upgrading via the '/Users/Rrando/Documents/GitHub/mlops-platform-spike-library/bugbug/mlflow/venv/bin/python3 -m pip install --upgrade pip' command.
